## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [ ]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [1]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd
import numpy as np


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [2]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [3]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [ ]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.0_RHOBTB2_direct.json') as inf:
    query1 = json.load(inf)

In [ ]:
printjson(query1)

In [ ]:
kcresult = submit_to_ars(query1)

In [ ]:
#retrieve_ars_results(kcresult)

In [14]:
kcresult = 'b3a39c22-e315-4e89-a642-1501d13461c2'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

#check_sheet.set_index('Workflow', inplace=True, drop=True)

In [15]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,D.1_parkinsons-crohns.json,NCBIGene:120892,NaN,n01,include,NaN,LRRK2
9,D.1_parkinsons-crohns.json,NCBIGene:11315,NaN,n01,include,NaN,PARK7


In [13]:
if 'D.1_parkinsons-crohns.json' in list(check_sheet.Workflow):
    print('True')

In [16]:
if 'D.1_parkinsons-crohns.json' in list(check_sheet.Workflow):
    dfy = check_sheet[check_sheet['Workflow']== 'D.1_parkinsons-crohns.json']
dfy.reset_index(drop=True)

for index,val in enumerate(dfy.Curie):
    print(index,val)
    node_num = dfy.iloc[index][3]
    query_id = val
    if np.isnan(dfy.iloc[index][2]) == True:
        len_check = 2000
    else:
        len_check = dfy.iloc[index][2]

    print(index, query_id, node_num, len_check)

0 NCBIGene:120892
0 NCBIGene:120892 n01 2000
1 NCBIGene:11315
1 NCBIGene:11315 n01 2000
2 NCBIGene:110357
2 NCBIGene:110357 n01 2000


In [ ]:
dfy

In [ ]:
if np.isnan(dfy.iloc[0][2]) == True:
    print('walla')

In [ ]:
check_sheet.columns

In [17]:
results = {}
dictionary = {}
dictionary_2 = {}
dict3 = {}
for child in j['children']:
    print(child['status'])
    error_code = child['code']

    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            
            child_response = requests.get(child_url).json()
            nresults = len(child_response['fields']['data']['message']['results'])
            
            if nresults > 0:
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                
            
                if 'D.1_parkinsons-crohns.json' in list(check_sheet.Workflow):
                    dfy = check_sheet[check_sheet['Workflow']== 'D.1_parkinsons-crohns.json']

                dfy.reset_index(drop=True)



                for index,curie_id in enumerate(dfy.Curie):
                    print(index,curie_id)
                    node_num = dfy.iloc[index][3]
                    query_id = curie_id
                    if np.isnan(dfy.iloc[index][2]) == True:
                        len_check = len(child_response['fields']['data']['message']['results'])
                    else:
                        len_check = dfy.iloc[index][2]

                    #print(node_num, query_id, len_check)

                    locs = []
                    for x, val in enumerate(child_response['fields']['data']['message']['results']):
                        #print(val)

                        if x < len_check:

                            if query_id in val['node_bindings'][node_num][0]['id']:
                                locs.append(x)
                    if not locs:
                        check_result = f'False'
                        print(check_result)
                        #pass
                    else:
                        check_result = f'True'
                        print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                    dict3[curie_id] = check_result
            #else:
                #continue
            
            if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                        edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                        test_att_values =[]
                        for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            #print(val)

                            for tx in edge_ex[val]['attributes']:
                                if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :


                                    value_att = tx['value']

                                    test_att_values.append(value_att)
                                    test_att = set(flatten_list(test_att_values))


                                    dictionary_2[child['actor']['agent']] = test_att
                #else:
                    #dictionary_2[child['actor']['agent']] = [] 
            #else:
               # dictionary_2[child['actor']['agent']] = []

        except Exception as e:
            nresults=0
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []



    elif child['status'] == 'Error':
        nresults=0
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            child_response = requests.get(child_url).json()
            results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            #dictionary_2[child['actor']['agent']] = []
        except Exception as e:
            #print(e)
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []


    else:
        nresults = 0
        #dictionary_2[child['actor']['agent']] = []

    dictionary['pk_id'] =  pk  

    if ((child['status'] == 'Done') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'No Results: ' + str(error_code)
        #test =  [child['actor']['agent'], 'No Results']
    elif ((child['status'] == 'ARS Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'ARS Error: ' + str(error_code)
    elif ((child['status'] == 'Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'Error: ' + str(error_code)
        #test =  [child['actor']['agent'], 'ARS Error']
    elif ((child['status'] == 'Done') & (nresults != 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Results: ' + str(error_code) + str(dict3)
    elif ((child['status'] == 'Unknown') & (nresults == 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Unknown: ' + str(error_code) 

    print(child['actor']['agent'], child['status'], nresults)
    #test =  [child['actor']['agent'], child['status'], nresults]
    #test2.append(test)

Done
kp-chp Done 0
Done
kp-icees-dili ARS Error 0
Done
kp-openpredict Done 0
Done
kp-cam Done 0
Done
ara-explanatory Done 0
Error
ara-bte Error 0
Error
ara-arax Error 0
Error
ara-unsecret Error 0
Error
ara-aragorn Error 0
Error
ara-robokop Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
0 NCBIGene:120892
curie id: NCBIGene:120892 : INCLUDED at postion N == [0] on n01
1 NCBIGene:11315
curie id: NCBIGene:11315 : INCLUDED at postion N == [2] on n01
2 NCBIGene:110357
False
ara-improving Done 22
Done
kp-icees ARS Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Unknown
ara-ncats Unknown 0
Error
kp-cohd Error 0


In [ ]:
dictionary_2

In [19]:
dictionary

{'pk_id': 'https://arax.ncats.io/?source=ARS&id=b3a39c22-e315-4e89-a642-1501d13461c2',
 'kp-chp': 'No Results: 200',
 'kp-icees-dili': 'ARS Error: 200',
 'kp-openpredict': 'No Results: 200',
 'kp-cam': 'No Results: 200',
 'ara-explanatory': 'No Results: 200',
 'ara-bte': 'Error: 598',
 'ara-arax': 'Error: 598',
 'ara-unsecret': 'Error: 598',
 'ara-aragorn': 'Error: 598',
 'ara-robokop': 'Error: 500',
 'kp-genetics': 'Unknown: 503',
 'kp-molecular': 'Error: 501',
 'ara-improving': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",
 'kp-icees': 'ARS Error: 200',
 'kp-textmining': 'No Results: 200',
 'ara-aragorn-exp': 'Error: 404',
 'ara-ncats': 'Unknown: 503',
 'kp-cohd': 'Error: 400'}

In [18]:
dict3

{'NCBIGene:120892': 'True',
 'NCBIGene:11315': 'True',
 'NCBIGene:110357': 'False'}

In [ ]:
'test' + str(dict3)